In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Concatenate, Attention, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

2024-08-30 15:43:01.941592: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-30 15:43:02.320417: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-30 15:43:02.416492: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-30 15:43:03.169841: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-30 15:43:06.516270: W tensorflow/comp

In [2]:
latent_dim = 256
num_samples = 10000

In [3]:
# Step 1: Load and preprocess the data
# Load dataset (replace with actual path)
data_path = 'dataset/eng_to_hindi.xlsx'  # Replace with actual path
df= pd.read_excel(data_path)

In [4]:
# Randomly sample 10,000 rows
#df = DF.sample(n=10000)

# Select the first 10,000 rows
#df = DF.head(15000)

# Select the last 10,000 rows
#df = DF.tail(15000)

In [5]:
english_sentences = df['English words/sentences'].tolist()
hindi_sentences = df['Unnamed: 1'].tolist()


In [7]:
# Preprocess data

# Add start and end tokens to Hindi sentences
hindi_sentences = ['\t ' + sentence + ' \n' for sentence in hindi_sentences]

# Tokenization and sequence padding
num_words = 10000  # Maximum number of words to keep in tokenizer
max_sequence_length = 50  # Maximum sequence length for padding

In [8]:
# English tokenizer
english_tokenizer = Tokenizer(num_words=num_words, filters=' ')
english_tokenizer.fit_on_texts(english_sentences)
english_sequences = english_tokenizer.texts_to_sequences(english_sentences)
english_sequences_padded = pad_sequences(english_sequences, maxlen=max_sequence_length, padding='post')

In [9]:
# Hindi tokenizer
hindi_tokenizer = Tokenizer(num_words=num_words, filters='')
hindi_tokenizer.fit_on_texts(hindi_sentences)
hindi_sequences = hindi_tokenizer.texts_to_sequences(hindi_sentences)
hindi_sequences_padded = pad_sequences(hindi_sequences, maxlen=max_sequence_length, padding='post')

In [10]:
hindi_sequences_padded.shape

(175621, 50)

In [11]:
hindi_sequences_padded[0]

array([   1, 8954,    2,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0], dtype=int32)

In [12]:
# Get the total number of unique tokens
num_encoder_tokens = len(english_tokenizer.word_index) + 1
num_decoder_tokens = len(hindi_tokenizer.word_index) + 1

# Define input and target data
encoder_input_data = english_sequences_padded
decoder_input_data = hindi_sequences_padded[:, :-1]
decoder_target_data = hindi_sequences_padded[:, 1:]

# Split data into training and validation sets
encoder_input_train, encoder_input_val, decoder_input_train, decoder_input_val, decoder_target_train, decoder_target_val = train_test_split(
    encoder_input_data, decoder_input_data, decoder_target_data, test_size=0.2
)

In [13]:
# Step 2: Define the model with bidirectional LSTM and attention

from tensorflow.keras.layers import Dot, Activation

# Encoder
latent_dim = 256
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_encoder_tokens, latent_dim)(encoder_inputs)
encoder_bi_lstm = Bidirectional(LSTM(latent_dim, return_sequences=True, return_state=True))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_bi_lstm(encoder_embedding)
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(num_decoder_tokens, latent_dim * 2)(decoder_inputs)
decoder_lstm = LSTM(latent_dim * 2, return_sequences=True, return_state=True)
decoder_lstm_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

# Luong attention mechanism
score_dot_product = Dot(axes=[2, 2])([decoder_lstm_outputs, encoder_outputs])  # Calculate the dot-product score
attention_weights = Activation('softmax')(score_dot_product)  # Softmax normalization to get attention weights
context_vector = Dot(axes=[2, 1])([attention_weights, encoder_outputs])  # Context vector is a weighted sum of encoder outputs

# Concatenate context vector with decoder LSTM outputs
decoder_combined_context = Concatenate(axis=-1)([context_vector, decoder_lstm_outputs])

# Dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_combined_context)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary to see the structure
model.summary()


2024-08-30 15:44:10.959796: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9462 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, None, 256) │  6,563,840 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ [(None, None,     │  1,050,624 │ embedding[0][0]   │
│ (Bidirectional)     │ 512), (None,      │            │                   │
│                     │ 256), (None,      │            │                   │
│                     │ 256), (None,      │            │                   │
│                     │ 256), (None,      │            │                   │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, None, 512) │  9,863,168 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 512)       │          0 │ bidirectional[0]… │
│ (Concatenate)       │                   │            │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 512)       │          0 │ bidirectional[0]… │
│ (Concatenate)       │                   │            │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, None,     │  2,099,200 │ embedding_1[0][0… │
│                     │ 512), (None,      │            │ concatenate[0][0… │
│                     │ 512), (None,      │            │ concatenate_1[0]… │
│                     │ 512)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot (Dot)           │ (None, None,      │          0 │ lstm_1[0][0],     │
│                     │ None)             │            │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, None,      │          0 │ dot[0][0]         │
│ (Activation)        │ None)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_1 (Dot)         │ (None, None, 512) │          0 │ activation[0][0], │
│                     │                   │            │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, None,      │          0 │ dot_1[0][0],      │
│ (Concatenate)       │ 1024)             │            │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None,      │ 19,745,600 │ concatenate_2[0]… │
│                     │ 19264)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 39,322,432 (150.00 MB)

 Trainable params: 39,322,432 (150.00 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Step 3: Train the model
batch_size = 64
epochs = 50

model.fit(
    [encoder_input_train, decoder_input_train],
    decoder_target_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=([encoder_input_val, decoder_input_val], decoder_target_val)
)

Epoch 1/50


2024-08-30 15:45:07.315263: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


2196/2196 ━━━━━━━━━━━━━━━━━━━━ 342s 153ms/step - accuracy: 0.8532 - loss: 1.0776 - val_accuracy: 0.8724 - val_loss: 0.7989
Epoch 2/50
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 338s 154ms/step - accuracy: 0.8785 - loss: 0.7575 - val_accuracy: 0.8854 - val_loss: 0.6869
Epoch 3/50
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 337s 153ms/step - accuracy: 0.8920 - loss: 0.6501 - val_accuracy: 0.8951 - val_loss: 0.6113
Epoch 4/50
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 337s 154ms/step - accuracy: 0.9014 - loss: 0.5741 - val_accuracy: 0.9051 - val_loss: 0.5473
Epoch 5/50
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 338s 154ms/step - accuracy: 0.9092 - loss: 0.5151 - val_accuracy: 0.9119 - val_loss: 0.4988
Epoch 6/50
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 338s 154ms/step - accuracy: 0.9165 - loss: 0.4616 - val_accuracy: 0.9175 - val_loss: 0.4575
Epoch 7/50
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 338s 154ms/step - accuracy: 0.9222 - loss: 0.4218 - val_accuracy: 0.9223 - val_loss: 0.4235
Epoch 8/50
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 339s 154ms/step - accuracy: 0.9

In [16]:
from tensorflow.keras.layers import Dot, Activation

# Step 4: Define inference models for prediction

# Encoder model for inference
encoder_model = Model(encoder_inputs, [encoder_outputs, state_h, state_c])

# Decoder model for inference
decoder_state_input_h = Input(shape=(latent_dim * 2,))
decoder_state_input_c = Input(shape=(latent_dim * 2,))
decoder_hidden_state_input = Input(shape=(None, latent_dim * 2))  # Encoder outputs

# Get decoder LSTM outputs
decoder_lstm_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=[decoder_state_input_h, decoder_state_input_c]
)

# Luong Attention mechanism
attention_scores = Dot(axes=[2, 2])([decoder_lstm_outputs, decoder_hidden_state_input])  # dot product
attention_scores = Activation('softmax')(attention_scores)  # apply softmax to get attention weights

# Compute context vector
context_vector = Dot(axes=[2, 1])([attention_scores, decoder_hidden_state_input])  # weighted sum

# Concatenate context vector and decoder LSTM outputs
decoder_combined_context = Concatenate(axis=-1)([context_vector, decoder_lstm_outputs])

# Pass combined context to the dense layer
decoder_outputs = decoder_dense(decoder_combined_context)

# Define decoder model for inference
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input, decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs] + [state_h, state_c]
)


In [22]:
# Step 5: Function for decoding sequences
def decode_sequence(input_seq):
    # Encode the input as state vectors
    encoder_output, h, c = encoder_model.predict(input_seq,verbose=0)

    # Generate empty target sequence of length 1
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = hindi_tokenizer.word_index['\t']

    # Sampling loop for a batch of sequences
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + [encoder_output, h, c],verbose=0)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = hindi_tokenizer.index_word[sampled_token_index]
        decoded_sentence += sampled_char

        # Add a space if the current character is not a space and the next character is not a punctuation
        if sampled_char not in (' ', '\n') and len(decoded_sentence) > 1 and not decoded_sentence[-2] in (' ', '\n'):
            decoded_sentence += ' '  
        # Exit condition: either hit max length or find stop character
        if (sampled_char == '\n' or len(decoded_sentence) > max_sequence_length):
            stop_condition = True

        # Update the target sequence (length 1)
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        h, c = [h, c]

    return decoded_sentence

In [23]:
# Test the model
test_sentence = "I spoke slowly so that they could understand me."
test_sentence_seq = english_tokenizer.texts_to_sequences([test_sentence])
test_sentence_padded = pad_sequences(test_sentence_seq, maxlen=max_sequence_length, padding='post')
decoded_sentence = decode_sequence(test_sentence_padded)
print('Decoded Sentence:', decoded_sentence)

KeyError: 'Exception encountered when calling Functional.call().\n\n\x1b[1m140014886984128\x1b[0m\n\nArguments received by Functional.call():\n  • inputs=tf.Tensor(shape=(1, 50), dtype=int32)\n  • training=False\n  • mask=None'

In [20]:
# Assuming encoder_inputs was defined with shape (None, max_sequence_length)
print('Encoder Input Shape:', encoder_model.input_shape)
print('Test Sentence Padded Shape:', test_sentence_padded.shape)


Encoder Input Shape: (None, None)
Test Sentence Padded Shape: (1, 50)


In [21]:
def decode_sequence(input_seq):
    # Ensure input_seq has the right shape
    if len(input_seq.shape) == 1:
        input_seq = np.expand_dims(input_seq, axis=0)

    # Encode the input as state vectors
    encoder_output, h, c = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1
    target_seq = np.zeros((1, 1))

    # Populate the first character of target sequence with the start character
    target_seq[0, 0] = target_tokenizer.word_index['<start>']

    # Sampling loop for a batch of sequences
    decoded_sentence = ''
    while True:
        output_tokens, h, c = decoder_model.predict(
            [target_seq, encoder_output, h, c]
        )

        # Sample a token and add the corresponding word to decoded_sentence
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_target_word_index[sampled_token_index]
        decoded_sentence += ' ' + sampled_word

        # Exit condition: either hit max length or find stop character.
        if sampled_word == '<end>' or len(decoded_sentence) > max_decoder_seq_length:
            break

        # Update the target sequence (length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        h, c = h, c

    return decoded_sentence

# Test the function with proper input
test_sentence = 'Your test sentence here.'
test_sentence_seq = english_tokenizer.texts_to_sequences([test_sentence])
test_sentence_padded = pad_sequences(test_sentence_seq, maxlen=max_sequence_length, padding='post')
decoded_sentence = decode_sequence(test_sentence_padded)
print('Decoded Sentence:', decoded_sentence)


KeyError: 'Exception encountered when calling Functional.call().\n\n\x1b[1m140014886984128\x1b[0m\n\nArguments received by Functional.call():\n  • inputs=tf.Tensor(shape=(1, 50), dtype=int32)\n  • training=False\n  • mask=None'